# Project runner
Imports the latest project version and runs the specified workflow

In [8]:
import nuclio

In [9]:
# nuclio: start-code

In [10]:
from mlrun import load_project
from mlrun import mlconf
import json
import os

In [22]:
def init_context(context):
    setattr(context, 'hub_url', os.getenv('hub_url', None))

In [37]:
def handler(context, event: nuclio.Event):
    """Imports the latest project version and runs the 
    specified workflow
    """
    details = json.loads(event.body)
    context.logger.info(type(details))
    context.logger.info(details)
    project_url = details['project_url']
    workflow = details['workflow']
    artifact_path = details.get('artifact_path', None)
    hub_url = details.get('hub_url', None)

    if context.hub_url:
        mlconf.hub_url = context.hub_url
    elif hub_url:
        mlconf.hub_url = hub_url

    project= load_project(os.path.abspath('./loaded_project'), url=project_url)
    project.run(name=workflow,
                arguments={},
                artifact_path=artifact_path,
                watch=False)

In [32]:
# nuclio: end-code

In [38]:
import json

event_details = {'project_url': '/User/demo-network-operations/project.yaml',
                 'workflow': 'main',
                 'hub_url': '/User/functions/{name}/function.yaml'}

In [ ]:
init_context(context)
event = nuclio.Event(body=json.dumps(event_details))
out = handler(context, event)
out